In [1]:
#Necessary imports for plotting, pandas, math operations, and multiple print statements.
import pandas as pd
from scipy.stats import chi2, chi2_contingency
from IPython.display import Image
import matplotlib.pyplot as plt
from math import pi
import numpy as np
import seaborn as sns
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [10]:
# read cleaned dataset
df = pd.read_csv("datasets/cleanest.csv")

main_df = df[["originalTitle", "Scripts", "genres"]]
main_df = main_df[main_df["Scripts"].str.contains("<NA>") == False].reset_index().drop("index", axis=1)
display(main_df)

# splitting genres
# df_split["genres"] = df_split["genres"].str.split(",")
# df_split = df_split.explode("genres")

# # convert "genres" from categorical to quantitative
# df_split["genres_num"] = df_split["genres"].map({'Comedy': 0, 'Fantasy': 1, 'Romance': 2, 'Drama': 3, 
#                                                                    'Thriller': 4, 'Action': 5, 'Crime': 6, 'Adventure': 7, 
#                                                                    'Family': 8, 'Mystery': 9, 'Biography': 10, 'Music': 11,
#                                                                    'Horror': 12, 'Sci-Fi': 13, 'Animation': 14,  'Sport': 15, 
#                                                                    'History': 16,  'Western': 17, 'War': 18,  'Musical': 19, 
#                                                                    'Documentary': 20
#                                                                    })

# display(scripts_basics_df[["originalTitle", "Scripts", "genres", "genres_num"]])
# display(scripts_basics_df["genres"].unique())
# display(scripts_basics_df["genres"].unique().size)
display(main_df["genres"].value_counts())

,originalTitle,Scripts,genres
0,Kate & Leopold,Time Time proposed fourth dimension And yet mo...,"Comedy,Fantasy,Romance"
1,Backdraft,font font P font P font Pr font font font font...,"Action,Crime,Drama"
2,Barton Fink,Kissing goodbye These four walls six flights s...,"Comedy,Drama,Thriller"
3,Bill & Ted's Bogus Journey,SUSPENSEFUL MUSIC PLAYING It time reached seco...,"Adventure,Comedy,Fantasy"
4,Billy Bathgate,What think Irving He makes cheap dago move Wei...,"Crime,Drama,Thriller"
...,...,...,...
2069,Hickok,No keep goddamn head right You hear Take gunne...,Western
2070,Lost in London,The Venom 5 million GTD Poker Tourney Download...,"Comedy,Drama"
2071,A Bad Moms Christmas,Ill blue Christmas Without Ill blue alright Wi...,Comedy
2072,SPF-18,real life movies get little mixed Just ask Sag...,"Drama,Romance,Sport"


Comedy,Drama,Romance       119
Comedy,Drama               102
Action,Crime,Drama          82
Comedy,Romance              80
Drama,Romance               73
                          ... 
Action,Comedy,Mystery        1
Mystery,Sci-Fi,Thriller      1
Crime,Drama,Sport            1
Drama,Horror,Musical         1
Western                      1
Name: genres, Length: 229, dtype: int64

In [3]:
# tf idf
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

tfidf = TfidfVectorizer()
response = tfidf.fit_transform(main_df["Scripts"])
feature_names = tfidf.get_feature_names()
# vectorized_df = pd.DataFrame(response.toarray()).set_axis([feature_names], axis=1).set_axis(training_df["originalTitle"], axis=0)
vectorized_df = pd.DataFrame(response.toarray()).set_axis(feature_names, axis=1)
vectorized_df

,00,000,0000,00000,00001,0002,000500010000,001,0010,0012,...,zounds,zs,zucchini,zulu,zulus,zuni,zurich,zwieback,zygote,zzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2069,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2071,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2072,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# edit vectorized dataframe
vectorized_df.insert(0, "movie_genres", main_df["genres"].copy())
vectorized_df.insert(0, "movie_titles", main_df["originalTitle"].copy())
vectorized_df

,movie_titles,movie_genres,00,000,0000,00000,00001,0002,000500010000,001,...,zounds,zs,zucchini,zulu,zulus,zuni,zurich,zwieback,zygote,zzz
0,Kate & Leopold,"Comedy,Fantasy,Romance",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Backdraft,"Action,Crime,Drama",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Barton Fink,"Comedy,Drama,Thriller",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bill & Ted's Bogus Journey,"Adventure,Comedy,Fantasy",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Billy Bathgate,"Crime,Drama,Thriller",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2069,Hickok,Western,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2070,Lost in London,"Comedy,Drama",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2071,A Bad Moms Christmas,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2072,SPF-18,"Drama,Romance,Sport",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# splitting genres for classifying purposes
# - training set has normal genres data
# - testing set has splitted genres data

# divide into train and test
vectorized_train = vectorized_df.loc[:500].copy()
vectorized_test = vectorized_df.loc[501:700].copy()

# splitting genres only for train
vectorized_train["movie_genres"] = vectorized_train["movie_genres"].squeeze().str.split(",")
vectorized_train = vectorized_train.explode("movie_genres")
vectorized_train

d = {'Comedy': 0, 'Fantasy': 1, 'Romance': 2, 'Drama': 3, 'Thriller': 4, 'Action': 5, 'Crime': 6, 'Adventure': 7, 
     'Family': 8, 'Mystery': 9, 'Biography': 10, 'Music': 11, 'Horror': 12, 'Sci-Fi': 13, 'Animation': 14,  
     'Sport': 15, 'History': 16,  'Western': 17, 'War': 18,  'Musical': 19, 'Documentary': 20
}

# def numerize_genre(x):
#     x.map(d)
#     return        

# convert "genres" from categorical to quantitative
vectorized_train["movie_genres"] = vectorized_train["movie_genres"].map(d)
vectorized_train

# vectorized_test["movie_genres"] = vectorized_test["movie_genres"].squeeze().str.split(",")
# vectorized_test
# vectorized_test["movie_genres"] = vectorized_test["movie_genres"].apply(lambda x : map(numerize_genre, x))
# vectorized_test

,movie_titles,movie_genres,00,000,0000,00000,00001,0002,000500010000,001,...,zounds,zs,zucchini,zulu,zulus,zuni,zurich,zwieback,zygote,zzz
0,Kate & Leopold,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,Kate & Leopold,Fantasy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,Kate & Leopold,Romance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Backdraft,Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Backdraft,Crime,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Hideous Kinky,Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
499,Hideous Kinky,Romance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
500,Arlington Road,Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
500,Arlington Road,Crime,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,movie_titles,movie_genres,00,000,0000,00000,00001,0002,000500010000,001,...,zounds,zs,zucchini,zulu,zulus,zuni,zurich,zwieback,zygote,zzz
0,Kate & Leopold,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,Kate & Leopold,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,Kate & Leopold,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Backdraft,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Backdraft,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Hideous Kinky,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
499,Hideous Kinky,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
500,Arlington Road,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
500,Arlington Road,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
vectorized_test_temp = vectorized_test.copy()
vectorized_test_temp["movie_genres"] = vectorized_test_temp["movie_genres"].squeeze().str.split(",")
vectorized_test_temp = vectorized_test_temp.explode("movie_genres")
vectorized_test_temp["movie_genres"] = vectorized_test_temp["movie_genres"].map(d)
vectorized_test_temp = vectorized_test_temp.groupby("movie_titles")["movie_genres"].apply(list).reset_index()
vectorized_test_temp
vectorized_test["movie_genres"] = vectorized_test_temp["movie_genres"].values
vectorized_test

,movie_titles,movie_genres
0,10 Things I Hate About You,"[0, 3, 2]"
1,15 Minutes,"[5, 6, 3]"
2,28 Days,"[0, 3]"
3,3000 Miles to Graceland,"[5, 0, 6]"
4,A Midsummer Night's Dream,"[0, 1, 2]"
...,...,...
195,What Women Want,"[0, 1, 2]"
196,Where the Heart Is,"[0, 3, 2]"
197,Wonder Boys,"[0, 3]"
198,You Can Count on Me,[3]


,movie_titles,movie_genres,00,000,0000,00000,00001,0002,000500010000,001,...,zounds,zs,zucchini,zulu,zulus,zuni,zurich,zwieback,zygote,zzz
501,Entrapment,"[0, 3, 2]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
502,Fight Club,"[5, 6, 3]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
503,The Astronaut's Wife,"[0, 3]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
504,Intolerable Cruelty,"[5, 0, 6]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
505,The Yards,"[0, 1, 2]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,Frailty,"[0, 1, 2]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
697,Murder by Numbers,"[0, 3, 2]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
698,America's Sweethearts,"[0, 3]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
699,The Rookie,[3],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# # splitting genres only for train
# vectorized_df["movie_genres"] = vectorized_df["movie_genres"].squeeze().str.split(",")
# vectorized_df = vectorized_df.explode("movie_genres")
# vectorized_df

# d = {'Comedy': 0, 'Fantasy': 1, 'Romance': 2, 'Drama': 3, 'Thriller': 4, 'Action': 5, 'Crime': 6, 'Adventure': 7, 
#      'Family': 8, 'Mystery': 9, 'Biography': 10, 'Music': 11, 'Horror': 12, 'Sci-Fi': 13, 'Animation': 14,  
#      'Sport': 15, 'History': 16,  'Western': 17, 'War': 18,  'Musical': 19, 'Documentary': 20
# }

# # convert "genres" from categorical to quantitative
# vectorized_df["movie_genres"] = vectorized_df["movie_genres"].map(d)
# vectorized_df

# vectorized_test = vectorized_df.loc[:300].copy()
# temp = vectorized_test.copy()
# # col_names = vectorized_test.columns.tolist()
# # col_names.remove('movie_titles')
# # col_names.remove('movie_genres')
# temp = temp.groupby("movie_titles")["movie_genres"].apply(list).reset_index()
# vectorized_test
# temp

In [8]:
from sklearn.neighbors import KNeighborsClassifier 
from numpy import dot
from numpy.linalg import norm

X_train = vectorized_train.copy().drop(["movie_titles", "movie_genres"], axis=1)
y_train = vectorized_train["movie_genres"]
X_test = vectorized_test.copy().drop(["movie_titles", "movie_genres"], axis=1)
y_test = vectorized_test["movie_genres"]

X_train
X_test
y_train
y_test

def cos_sim(x, y):
    # print(dot(x, y) / (norm(x) * norm(y)))
    return dot(x, y) / (norm(x) * norm(y))

# knn_model = KNeighborsClassifier(n_neighbors=10, weights='distance', metric=cos_sim)
knn_model = KNeighborsClassifier(n_neighbors=15, metric=cos_sim)
knn_model.fit(X=X_train, y=y_train)


,00,000,0000,00000,00001,0002,000500010000,001,0010,0012,...,zounds,zs,zucchini,zulu,zulus,zuni,zurich,zwieback,zygote,zzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
499,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,00,000,0000,00000,00001,0002,000500010000,001,0010,0012,...,zounds,zs,zucchini,zulu,zulus,zuni,zurich,zwieback,zygote,zzz
501,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
502,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
504,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
697,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
698,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


0      0
0      1
0      2
1      5
1      6
      ..
499    3
499    2
500    5
500    6
500    3
Name: movie_genres, Length: 1303, dtype: int64

501    [0, 3, 2]
502    [5, 6, 3]
503       [0, 3]
504    [5, 0, 6]
505    [0, 1, 2]
         ...    
696    [0, 1, 2]
697    [0, 3, 2]
698       [0, 3]
699          [3]
700          [0]
Name: movie_genres, Length: 200, dtype: object

KNeighborsClassifier(metric=<function cos_sim at 0x000002158F43C550>,
                     n_neighbors=15)

In [9]:
# pd.set_option('display.max_colwidth', None)

# - "predict" y values of X_train values
# - compare these predicted values to the actual values y_train to evaluate
from sklearn.metrics import mean_squared_error
test_preds = knn_model.predict(X_test)
test_preds
# mse = mean_squared_error(y_train, train_preds)
# mse

array([0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 0], dtype=int64)

In [17]:
# evaluate the performance of our model

# compute the error percentage of two arrays
# actual is an array of arrays of numbers
# predicted is an array of numbers
# return the number of correct predictions/total
def error_perc(actual, predicted):
    correct = 0;
    for x, y in zip(actual, predicted):
        for i in x:
            if y == i:
                correct = correct + 1
                # print(x, y)
                # print(str(correct) + ", " + str(len(actual)))
    return correct / len(actual)

ep = error_perc(y_test, test_preds)
ep


0.355